# Boosting large images plotting with [vaex](http://vaex.astro.rug.nl/)


## About this notebook

This notebook belongs to a series of small projects which aim is to evaluate the [Jupyter](http://jupyter.org/) ecosystem for science experiments control. The main idea is use the _Juypter notebook_ as a convergence platform in order to offer a fully featured environment to scientists. 

## Topic of the day

Use `vaex`for data binning and `bokeh` for display.

### First, we route bokeh outputs to notebook cells
This will also load BokehJS - the JavaScript part of bokeh.

In [ ]:
from bokeh.io import output_notebook 
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

### Import BokehSession & related classes (embedded server)

In [ ]:
from common.session import BokehSession

### Scanner class
Simulates a scanner delivering an image by bunches of N rows.  


In [ ]:
import time
import numpy as np

from IPython.display import clear_output

from bokeh.plotting import figure
from bokeh.plotting.figure import Figure
from bokeh.models.glyphs import Rect
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, Button
from bokeh.models.mappers import LinearColorMapper
from bokeh.palettes import Plasma256, Viridis256
from bokeh.layouts import row, layout, widgetbox


class Scanner(object):
    
    def __init__(self, imw=100, imh=100): 
        '''
        imw: image width
        imh: image height
        '''
        # scan progress: num of row added to the image at each iteration
        self._inc = 1
        # scan progress: row at which new data is injected at next iteration
        self._row_index = 0
        # scan image width and height
        self._iw, self._ih = imw, imh
        # image buffer (from which scan data is extracted - for simulation purpose)
        x, y = np.linspace(0, 10, imw), np.linspace(0, 10, imh)
        xx, yy = np.meshgrid(x, y) 
        self._data_source = np.sin(xx) * np.cos(yy)
        
    def empty_image(self):
        # produce an empty scanner image
        empty_img = np.empty((self._ih, self._iw))
        empty_img.fill(np.nan)
        return empty_img
         
    def image(self):
        # return 'current' scanner image (simulate scan progress)
        start = None
        end = self._row_index + self._inc
        s1, s2 = slice(start, end), slice(None)
        index = [0, s1, s2]
        image = self.empty_image()
        image[s1, s2] = self._data_source[s1, s2]
        self._row_index = end % self._ih
        return image
    
    def reset(self):
        # reset 'current' image
        self._row_index = 0
        
    @property
    def x_range(self):
        return (0, self._iw)

    @property
    def y_range(self):
        return (0, self._ih)
    
    @property
    def num_pixels(self):
        return self._iw * self._ih
    
    @property
    def inc(self):
        return self._inc
 
    @inc.setter
    def inc(self, inc):
        self._inc = max(1, inc)
  

### ScannerDisplay class
A user specialization of the `BokehSession` in charge of scanner image display. This also applies the `vaex` binning in case the scanner image size is above the specified threshold. 

The `BokehSession` simply hides the details related to the bokeh embdedded server and provides asynchronous (i.e. periodic) activity support.  

In [ ]:
  class ScannerDisplay(BokehSession):
    
    def __init__(self, imw=100, imh=100, ist=1e5, upp=0.5):
        '''
        imw: image width
        imh: image height
        ist: image size threshold above which vaex binning is applied
        upp: plot update period in seconds  
        '''
        BokehSession.__init__(self)
        # the underlying scanner
        self._scanner = Scanner(imw, imh)
        # image size above which vaex optimization is enabled
        self._image_size_threshold = ist
        # image plot update period in seconds
        self.callback_period = upp
        # bokeh column data source
        self._cds = None
        # suspend/resume button
        self._suspend_resume_button = None
        
    def __setup_cds(self):
        self._cds = ColumnDataSource(data=dict(img=[self._scanner.empty_image()]))
        return self._cds
    
    def __reset(self):
        self._cds.data.update(img=[self._scanner.empty_image()])
        self._scanner.reset()
        self.resume()
    
    def __on_update_period_change(self, attr, old, new):
        """called when the user changes the refresh period using the dedicated slider"""
        self.update_callback_period(new)

    def __on_slice_size_change(self, attr, old, new):
        """called when the user changes the slice size using the dedicated slider"""
        self._scanner.inc = int(new)
        
    def __suspend_resume(self): 
        """suspend/resume preriodic activity"""
        if self.suspended:
            self._suspend_resume_button.label = 'suspend'
            self.resume()
        else:
            self._suspend_resume_button.label = 'resume'
            self.pause()
        
    def __close(self):  
        """tries to cleanup everything properly"""
        # celear cell ouputs
        clear_output()
        # cleanup the session
        self.close()
        
    def setup_document(self):
        """setup the session document"""
        # close button
        rb = Button(label='reset')
        rb.on_click(self.__reset)
        # close button
        cb = Button(label='close')
        cb.on_click(self.__close)
        # suspend/resume button
        self._suspend_resume_button = Button(label='suspend')
        self._suspend_resume_button.on_click(self.__suspend_resume)
        # a slider to control the update period
        upp = Slider(start=0.1, end=2, step=0.01, value=self.callback_period, title="Updt.period [s]",)
        upp.on_change("value", self.__on_update_period_change)
        # a slider to control the update period
        max_val = max(1, self._scanner.y_range[1] / 10)
        inc = Slider(start=1, end=max_val, step=1, value=self._scanner.inc, title="Slice size [rows]",)
        inc.on_change("value", self.__on_slice_size_change)
        # the figure and its content
        f = figure(plot_width=400, plot_height=350, x_range=self._scanner.x_range, y_range=self._scanner.y_range)
        ikwargs = dict()
        ikwargs['x'] = 0
        ikwargs['y'] = 0
        ikwargs['dw'] = self._scanner.x_range[1]
        ikwargs['dh'] = self._scanner.y_range[1]
        ikwargs['image'] = 'img'
        ikwargs['source'] = self.__setup_cds()
        ikwargs['color_mapper'] = LinearColorMapper(Viridis256)
        f.image(**ikwargs)
        # widgets are placed into a dedicated layout
        w = widgetbox(upp, inc, rb, self._suspend_resume_button, cb)
        # arrange all items into a layout then add it to the document
        self.document.add_root(layout([[w, f]]), setter=self.id) 
        # start periodic activity
        self.resume()
    
    def periodic_callback(self):
        """periodic activity"""
        try:
            # get image from scanner
            image = self._scanner.image()
            # enable vaex binning in case scanner image size is above threshold
            if self._scanner.num_pixels >= self._image_size_threshold:
                print('apply_vaex_binning!')
                image = self.apply_vaex_binning(image)
            # update bokeh plot
            self._cds.data.update(img=[image])
        except Exception as e:
            print(e)
            
    def apply_vaex_binning(self, numpy_array_image):
        # 0. more args might be required
        # 1. convert from np array to vaex dataset: vaex.from_arrays(...)
        # 2. vaex_result = apply vaex binnning
        # 3. convert vaex_result back into a np array
        vaex_result_as_numpy_array = numpy_array_image
        return vaex_result_as_numpy_array

### Let's go...
Change image size playing with both `img_width` and `img_height`. In my case, the 'acceptable' performances limit is around 1000 x 500.

In [ ]:
# scanner image width
img_width = 100
# scanner image height
img_height = 100
# scanner image size above which we want to enable vaex binnning
img_size_threshold = 1e5
# instanciate than open scanner image display 
d = ScannerDisplay(img_width, img_height, img_size_threshold)
d.open()